<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [2]:
#!pip3 install twilio

In [3]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI 
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [4]:
query = 'cartagena'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=dc39efed40cd4437842232730230401&q=cartagena&days=1&aqi=no&alerts=no'

In [5]:
response = requests.get(url_clima).json()

In [6]:
response

{'location': {'name': 'Cartagena',
  'region': 'Bolivar Department',
  'country': 'Colombia',
  'lat': 10.4,
  'lon': -75.51,
  'tz_id': 'America/Bogota',
  'localtime_epoch': 1674417096,
  'localtime': '2023-01-22 14:51'},
 'current': {'last_updated_epoch': 1674416700,
  'last_updated': '2023-01-22 14:45',
  'temp_c': 31.0,
  'temp_f': 87.8,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 9.4,
  'wind_kph': 15.1,
  'wind_degree': 310,
  'wind_dir': 'NW',
  'pressure_mb': 1007.0,
  'pressure_in': 29.74,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 63,
  'cloud': 25,
  'feelslike_c': 35.9,
  'feelslike_f': 96.5,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 7.0,
  'gust_mph': 13.2,
  'gust_kph': 21.2},
 'forecast': {'forecastday': [{'date': '2023-01-22',
    'date_epoch': 1674345600,
    'day': {'maxtemp_c': 32.4,
     'maxtemp_f': 90.3,
     'mintemp_c': 23.4,
     'mintemp_f':

In [7]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [8]:
response['forecast']['forecastday'][0].keys()


dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [9]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [10]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1674363600,
 'time': '2023-01-22 00:00',
 'temp_c': 24.8,
 'temp_f': 76.6,
 'is_day': 0,
 'condition': {'text': 'Clear',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 11.9,
 'wind_kph': 19.1,
 'wind_degree': 46,
 'wind_dir': 'NE',
 'pressure_mb': 1009.0,
 'pressure_in': 29.79,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 82,
 'cloud': 8,
 'feelslike_c': 26.9,
 'feelslike_f': 80.4,
 'windchill_c': 24.8,
 'windchill_f': 76.6,
 'heatindex_c': 26.9,
 'heatindex_f': 80.4,
 'dewpoint_c': 21.6,
 'dewpoint_f': 70.9,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 19.5,
 'gust_kph': 31.3,
 'uv': 1.0}

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2023-01-22'

In [12]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [13]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [14]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [15]:
#recoresponserd
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [16]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<?, ?it/s]


In [17]:
datos[0]

('2023-01-22', 0, 'Clear', 24.8, 0, 0)

In [18]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-01-22,0,Clear,24.8,0,0
1,2023-01-22,1,Clear,24.6,0,0
2,2023-01-22,2,Clear,24.3,0,0
3,2023-01-22,3,Clear,24.1,0,0
4,2023-01-22,4,Clear,23.8,0,0
5,2023-01-22,5,Clear,23.6,0,0
6,2023-01-22,6,Clear,23.4,0,0
7,2023-01-22,7,Sunny,23.6,0,0
8,2023-01-22,8,Sunny,25.1,0,0
9,2023-01-22,9,Sunny,26.8,0,0


In [19]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [20]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia


In [21]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [22]:
df['Fecha'][0]

'2023-01-22'

In [23]:
df_rain

,Condicion
Hora,


In [24]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)+'Mensaje enviado con una configuracion nueva'

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-01-22 en cartagena es : \n\n\n Empty DataFrame\nColumns: [Condicion]\nIndex: []Mensaje enviado con una configuracion nueva'

In [25]:
PHONE_NUMBER

'+1 6693382710'

# Mensaje SMS desde Twilio

In [27]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages.create(
    #messaging_service_sid='MG19303578059b088770dcb78de598c706',
    body='\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
    from_=PHONE_NUMBER,
    to='+57 321 9168341')

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM7733e2914b22092beed3c36e26fca53e


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />